# The Randomized Benchmarking Protocol

In [ ]:
# python -m venv qiskit_env
 
#  qiskit_env\Scripts\activate  

# pip install qiskit 
# pip install jupyter
# pip install qiskit-aer  
# pip install matplotlib 
# pip install qiskit-ibm-runtime

# Randomized benchmarking is a technique used to assess the performance of quantum gates and circuits. 
# The goal is to evaluate how much errors or noise are introduced during quantum operations, 
# such as the application of quantum gates. It helps to measure the fidelity (accuracy) 
# of quantum operations without needing to know the exact state of the qubits.

# How It Works:
# Apply Random Gates: Random quantum gates are applied to the qubits.
# Measure the Outcome: After the gates, the qubits are measured.
# Repeat: This is done many times with different sequences of gates.
# Calculate Fidelity: The measurement results are compared to the expected outcome to find how close they are.
# Estimate Errors: The error rates of the quantum gates are calculated based on these results.





# Import necessary libraries
# ---------------------------
# numpy: Used for numerical operations, particularly for generating random values in the circuit and for computing fidelity.
# QuantumCircuit: Allows us to define and manipulate quantum circuits in Qiskit.
# transpile: Optimizes circuits for specific backends.
# Aer: Provides the Qiskit Aer simulator, allowing us to run quantum circuits locally on a simulator.
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer

# Define a function to generate a random quantum circuit
# ------------------------------------------------------
# This function generates a circuit with random rotations for each qubit.
# num_qubits: Number of qubits in the circuit.
# depth: Number of random rotation layers to apply.
def generate_random_circuit(num_qubits, depth):
    # Create a quantum circuit with num_qubits qubits and classical bits.
    circuit = QuantumCircuit(num_qubits, num_qubits)
    
    # Apply random single-qubit rotations for each depth layer.
    for _ in range(depth):
        for qubit in range(num_qubits):
            # Apply a random RX, RY, and RZ rotation to each qubit.
            circuit.rx(np.random.uniform(0, 2 * np.pi), qubit)
            circuit.ry(np.random.uniform(0, 2 * np.pi), qubit)
            circuit.rz(np.random.uniform(0, 2 * np.pi), qubit)
        
        # Add controlled-Z (CZ) gates between neighboring qubits to introduce entanglement.
        for qubit in range(num_qubits - 1):
            circuit.cz(qubit, qubit + 1)
    
    return circuit

# Define the randomized benchmarking function
# -------------------------------------------
# num_qubits: Number of qubits in the circuit.
# depths: List of depths to test for the benchmarking.
# num_sequences: Number of random sequences to generate and test for each depth.
# shots: Number of shots to estimate the success rate.
def randomized_benchmarking(num_qubits, depths, num_sequences, shots):
    # Use the statevector simulator for ideal fidelity calculation
    backend = Aer.get_backend('statevector_simulator')
    
    # Initialize a list to store the success rates for each depth
    results = []
    
    # Loop through each depth value
    for depth in depths:
        # Initialize a counter to accumulate success counts
        success_counts = 0
        
        # Generate and evaluate multiple random circuits for each depth
        for _ in range(num_sequences):
            # Step 1: Generate a random circuit and its inverse
            circuit = generate_random_circuit(num_qubits, depth)    # Generate a random circuit
            inverse_circuit = circuit.inverse()                     # Create the inverse circuit

            # Step 2: Apply the circuit and obtain the final statevector
            circuit_result = backend.run(circuit).result()          # Run the circuit on the simulator
            final_statevector = circuit_result.get_statevector()     # Retrieve the final statevector
            
            # Step 3: Apply the inverse circuit to return to the initial state
            inverse_result = backend.run(inverse_circuit).result()  # Run the inverse circuit on the simulator
            inverse_statevector = inverse_result.get_statevector()   # Retrieve the statevector after inverse

            # Step 4: Calculate the success rate based on fidelity
            # Fidelity is calculated as the overlap between final_statevector and inverse_statevector
            fidelity = np.abs(np.dot(final_statevector, inverse_statevector.conj())) ** 2
            success_counts += shots * (1 - fidelity)                # Update success count with (1 - fidelity)

        # Calculate average success rate for the current depth
        success_rate = success_counts / (num_sequences * shots)
        results.append(success_rate)  # Store the success rate for this depth
    
    return results

# Parameters for benchmarking
num_qubits = 2                    # Number of qubits
depths = [1, 2, 3, 4]             # List of depths to test
num_sequences = 100               # Number of random sequences per depth
shots = 1024                      # Number of shots per circuit

# Execute the randomized benchmarking protocol
results = randomized_benchmarking(num_qubits, depths, num_sequences, shots)
print("Success rates at each depth:", results)


# Random Circuits: These are circuits that apply random operations (like rotations and entangling gates) to qubits.

# Inverse Circuits: After applying a random circuit, 
# an inverse of the circuit is applied to return to the original state.

# Fidelity: This measures how close the final state of the system is to the initial state 
# (after applying the random circuit and its inverse). 
# A perfect fidelity means the final state is exactly the same as the initial state.


ImportError: Qiskit is installed in an invalid environment that has both Qiskit >=1.0 and an earlier version. You should create a new virtual environment, and ensure that you do not mix dependencies between Qiskit <1.0 and >=1.0. Any packages that depend on 'qiskit-terra' are not compatible with Qiskit 1.0 and will need to be updated. Qiskit unfortunately cannot enforce this requirement during environment resolution. See https://qisk.it/packaging-1-0 for more detail.